In [21]:
from math import sin, cos, sqrt, atan2, radians
import pandas as pd
import requests
import json
import io

zip_code_original = '11429'
zip_distance_api_key = 'OSOjg5nPLvlsrD990zDvW0PbUeCG0GtDDcHCZsXNiv7tWtuutg1WJBtPsZv7T8yj'
url_zipcode_csv = "https://gist.githubusercontent.com/erichurst/7882666/raw/5bdc46db47d9515269ab12ed6fb2850377fd869e/US%2520Zip%2520Codes%2520from%25202013%2520Government%2520Data"

In [22]:
get_url = requests.get(url_zipcode_csv).content
df = pd.read_csv(io.StringIO(get_url.decode('cp1252')))
df['ZIP'] = df['ZIP'].map('{:0>5}'.format)  
df.head(2)

,ZIP,LAT,LNG
0,00601,18.180555,-66.749961
1,00602,18.361945,-67.175597


In [50]:
R = 10000
distance_list = []
[LAT1] = df.loc[df['ZIP'] == '11429']['LAT'].values
[LON1] = df.loc[df['ZIP'] == '11429']['LNG'].values
LAT2 = df['LAT'].to_list()
LON2 = df['LNG'].to_list()
lat1 = radians(LAT1)
lon1 = radians(LON1)
for la2, lo2 in zip(LAT2, LON2):
    lat2 = radians(la2)
    lon2 = radians(lo2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    distance_list.append(distance)
df['distance_from_zipcode_input'] = distance_list
df.sort_values(by = 'distance_from_zipcode_input', inplace = True)
df.head()

,ZIP,LAT,LNG,distance_from_zipcode_input
2918,11429,40.709766,-73.738653,0.000000
2917,11428,40.721016,-73.742245,2.020176
2900,11411,40.694021,-73.736216,2.766875
2901,11412,40.698095,-73.758986,3.374453
2916,11427,40.730904,-73.745661,3.803960


In [55]:
store_num_list = []
while len(store_num_list) < 3:
    for zipcode in df['ZIP'].to_list():
        response_rite_aid_store_finder = requests.get(f'https://www.riteaid.com/services/ext/v2/stores/getStores?address={zipcode}&attrFilter=PREF-112&fetchMechanismVersion=2&radius=500')
        get_req = response_rite_aid_store_finder.json()
        store_list = get_req['Data']['stores']
        store_numbers_list = [store['storeNumber'] for store in store_list]
        for store_num in store_numbers_list:
            response_get_slots = requests.get(f'https://www.riteaid.com/services/ext/v2/vaccine/checkSlots?storeNumber={store_num}')
            get_request = response_get_slots.json()
            if get_request["Data"]["slots"]["1"] == True:
        #         print(store_num)
        #         print(get_request)
                store_num_list.append(store_num)
            print(store_num)
            print(get_request)
store_num_list

3865
{'Data': {'slots': {'1': False, '2': False}}, 'Status': 'SUCCESS', 'ErrCde': None, 'ErrMsg': None, 'ErrMsgDtl': None}
1809
{'Data': {'slots': {'1': False, '2': False}}, 'Status': 'SUCCESS', 'ErrCde': None, 'ErrMsg': None, 'ErrMsgDtl': None}
4552
{'Data': {'slots': {'1': False, '2': False}}, 'Status': 'SUCCESS', 'ErrCde': None, 'ErrMsg': None, 'ErrMsgDtl': None}
4277
{'Data': {'slots': {'1': False, '2': False}}, 'Status': 'SUCCESS', 'ErrCde': None, 'ErrMsg': None, 'ErrMsgDtl': None}
2771
{'Data': {'slots': {'1': False, '2': False}}, 'Status': 'SUCCESS', 'ErrCde': None, 'ErrMsg': None, 'ErrMsgDtl': None}
4836
{'Data': {'slots': {'1': False, '2': False}}, 'Status': 'SUCCESS', 'ErrCde': None, 'ErrMsg': None, 'ErrMsgDtl': None}
1203
{'Data': {'slots': {'1': False, '2': False}}, 'Status': 'SUCCESS', 'ErrCde': None, 'ErrMsg': None, 'ErrMsgDtl': None}
10620
{'Data': {'slots': {'1': False, '2': False}}, 'Status': 'SUCCESS', 'ErrCde': None, 'ErrMsg': None, 'ErrMsgDtl': None}
3626
{'Data': {

From cffi callback <function _verify_callback at 0x0000021D71FD8EE8>:
Traceback (most recent call last):
  File "C:\Users\lilyx\Anaconda3\lib\site-packages\OpenSSL\SSL.py", line 306, in wrapper
    @wraps(callback)
KeyboardInterrupt


SSLError: HTTPSConnectionPool(host='www.riteaid.com', port=443): Max retries exceeded with url: /services/ext/v2/vaccine/checkSlots?storeNumber=4202 (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))

In [52]:
response = requests.get('https://www.riteaid.com/services/ext/v2/vaccine/checkSlots?storeNumber=1496')
get_request = response.json()
if get_request["Data"]["slots"]["1"] == True:
    print(get_request)

{'Data': {'slots': {'1': True, '2': False}}, 'Status': 'SUCCESS', 'ErrCde': None, 'ErrMsg': None, 'ErrMsgDtl': None}
